In [57]:
#type:ignore
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import matplotlib as plt
from tensorflow.keras.applications import ResNet50, MobileNetV2  
from tensorflow.keras.models import Model
from tensorflow.keras import layers,optimizers
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
IMG_SIZE =224

In [58]:
df = pd.read_csv('labels.csv')

In [59]:
df.head()

,name,category,type,grade
0,1.jpg,0,4,3
1,2.jpg,0,4,3
2,3.jpg,0,2,2
3,4.jpg,0,2,1
4,5.jpg,0,1,3


In [60]:
image_paths = ['images/'+ name for name in df['name'].values]

In [61]:
y = df['type'].values

In [62]:
y_augmented = np.concatenate([y, y])

In [63]:
y_labels = to_categorical(y_augmented,num_classes=5)

In [64]:
y_labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [65]:
def load_image(image_path):
  img = cv2.imread(image_path)      # read image
  if img is None:
      print(f"Warning: Unable to load image at {image_path}")
      return None
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)       # convert to RGB
  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))    # resize for the model
  img = img/255.0      # normalize pixel values to between 0 and 1

  return img

In [66]:
def rotate_image(image_path):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)  # apply same preprocessing features as original image

  img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) 

  img = img/255.0

  return img

In [67]:
augmented_images = []

for path in image_paths:
    img = load_image(path)          # Original
    augmented_images.append(img)
    
    # Augment (rotate by 90 degrees)
    rotated_img = rotate_image(path)
    augmented_images.append(rotated_img)

# Convert to numpy array
X_augmented = np.array(augmented_images)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.3, random_state=42) # training and testing

In [69]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))  # loading model without top layers

In [70]:
#base_model.trainable = False  # Freezing layers to exclude from training

In [71]:
for layer in base_model.layers[-20:]:  # unfreezing last 20 layers to fine-tune
    layer.trainable = True

In [72]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

In [73]:
output_layer = Dense(5, activation='softmax', name='type')(x)

In [74]:
model = Model(inputs = base_model.input, outputs=output_layer)

In [75]:
model.compile(optimizer='adam', loss=
        'sparse_categorical_crossentropy', metrics=['accuracy'])

In [76]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,587,205 (9.87 MB)

 Trainable params: 2,553,093 (9.74 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [77]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test), verbose=1)

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6789 - loss: 1.2649 - val_accuracy: 0.7734 - val_loss: 5.9117
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.7763 - loss: 0.8116 - val_accuracy: 0.7734 - val_loss: 5.5533
Epoch 3/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.7610 - loss: 0.7511 - val_accuracy: 0.7734 - val_loss: 4.4201
Epoch 4/15
 6/32 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.7563 - loss: 0.7232

KeyboardInterrupt: 

In [78]:
model.save('corneal_ulcer_model.h5')